In [76]:
import os
import pandas as pd
import re
import geopandas as gpd
import numpy as np
from tqdm import tqdm

In [2]:
prev_data = pd.read_parquet('../data/second_departure_parquet_preprocessed/Данные_за_06.09-12.09/Тест_5hz_-2/Тест_5_Hz_-2_06.09.2022.parquet')

In [4]:
prev_data

,Наименование (device ID),Время отправки гео,bearing,Акселерометр,Гироскоп,Счетчик шагов,Скорость,"Точность, м",Долгота,Широта,Высота,Уровень заряда,timestamp_rownum,acc_x,acc_y,acc_z
0,40808752-2CEE-454B-B6E6-95EC9615999D,2022-09-06 08:14:50,293,"[0.13168335, 5.323807, 7.882904]",[],3,0.223073,30,37.602669,55.892841,187.9,41,1,0.131683,5.323807,7.882904
1,40808752-2CEE-454B-B6E6-95EC9615999D,2022-09-06 08:14:50,293,"[-1.4389191, 6.12825, 6.4655304]",[],3,0.223073,30,37.602669,55.892841,187.9,41,2,-1.438919,6.128250,6.465530
2,40808752-2CEE-454B-B6E6-95EC9615999D,2022-09-06 08:14:50,293,"[-1.2473755, 6.0516357, 6.6953735]",[],3,0.223073,30,37.602669,55.892841,187.9,41,3,-1.247375,6.051636,6.695373
3,40808752-2CEE-454B-B6E6-95EC9615999D,2022-09-06 08:14:51,293,"[-0.4046173, 5.3621063, 7.882904]",[],3,0.223073,30,37.602669,55.892841,187.9,41,1,-0.404617,5.362106,7.882904
4,40808752-2CEE-454B-B6E6-95EC9615999D,2022-09-06 08:14:51,293,"[-0.5195465, 5.4770355, 7.882904]",[],3,0.223073,30,37.602669,55.892841,187.9,41,2,-0.519547,5.477036,7.882904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161210,40808752-2CEE-454B-B6E6-95EC9615999D,2022-09-06 17:31:46,12,"[1.0510559, 4.6342773, 8.189362]",[],10833,0.216062,12,37.604434,55.886586,180.9,66,2,1.051056,4.634277,8.189362
161211,40808752-2CEE-454B-B6E6-95EC9615999D,2022-09-06 17:31:46,12,"[1.280899, 4.7491913, 8.419205]",[],10833,0.216062,12,37.604434,55.886586,180.9,66,3,1.280899,4.749191,8.419205
161212,40808752-2CEE-454B-B6E6-95EC9615999D,2022-09-06 17:31:46,12,"[1.2042847, 4.8641205, 8.45752]",[],10833,0.216062,12,37.604434,55.886586,180.9,66,4,1.204285,4.864121,8.457520
161213,40808752-2CEE-454B-B6E6-95EC9615999D,2022-09-06 17:31:46,12,"[1.3192139, 4.8258057, 8.763977]",[],10833,0.216062,12,37.604434,55.886586,180.9,66,5,1.319214,4.825806,8.763977


In [30]:
prev_cols = prev_data.columns

In [26]:
new_data = pd.read_parquet('../data/third_departure_parquet_preprocessed/raw/D4909E26-0469-4045-B426-211006C38915/01.12.2022.parquet')

In [27]:
new_data

,userid,Наименование,Время создания записи,Время отправки гео,bearing,charge,isPermissionGranted,Акселерометр,Гироскоп,Счетчик шагов,...,passive,"Точность, м",Долгота,Широта,Высота,Уровень заряда,timestamp_rownum,acc_x,acc_y,acc_z
0,2896,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:54.702,2022-12-01 14:03:35,274,False,True,"[9.241028, -1.1326599, -1.6559906]",[],21954,...,True,29,37.613327,55.894135,179.2,60,1,9.241028,-1.132660,-1.655991
1,2896,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:54.702,2022-12-01 14:03:36,274,False,True,"[10.447708, -3.0320587, -3.1914673]",[],21954,...,True,29,37.613327,55.894135,179.2,60,1,10.447708,-3.032059,-3.191467
2,2896,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:54.702,2022-12-01 14:03:36,274,False,True,"[7.4182434, -2.138214, -2.4093628]",[],21954,...,True,29,37.613327,55.894135,179.2,60,2,7.418243,-2.138214,-2.409363
3,2896,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:54.702,2022-12-01 14:03:36,274,False,True,"[11.8810425, -1.5572205, -3.9480438]",[],21954,...,True,29,37.613327,55.894135,179.2,60,3,11.881042,-1.557220,-3.948044
4,2896,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:54.702,2022-12-01 14:03:36,274,False,True,"[11.277695, -0.86450195, -2.7190094]",[],21957,...,True,29,37.613327,55.894135,179.2,60,4,11.277695,-0.864502,-2.719009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98445,2896,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:33.563,2022-12-01 14:03:31,109,False,True,"[10.138062, -0.8421631, -1.3686829]",[],21952,...,True,22,37.613377,55.894082,179.2,60,3,10.138062,-0.842163,-1.368683
98446,2896,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:33.563,2022-12-01 14:03:31,109,False,True,"[10.252975, -0.20370483, -1.2441864]",[],21952,...,True,22,37.613377,55.894082,179.2,60,4,10.252975,-0.203705,-1.244186
98447,2896,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:33.563,2022-12-01 14:03:31,102,False,True,"[10.252975, -0.20370483, -1.2441864]",[],21952,...,True,22,37.613391,55.894084,179.2,60,5,10.252975,-0.203705,-1.244186
98448,2896,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:33.563,2022-12-01 14:03:31,6,False,True,"[10.252975, -0.20370483, -1.2441864]",[],21952,...,True,25,37.613397,55.894129,179.2,60,6,10.252975,-0.203705,-1.244186


In [33]:
new_data.rename(columns={'Наименование':'Наименование (device ID)'}, inplace=True)

In [34]:
new_cols = new_data.columns

In [35]:
diff_columns = [col for col in new_cols if col not in prev_cols]

In [36]:
diff_columns

['userid',
 'Время создания записи',
 'charge',
 'isPermissionGranted',
 'step_count',
 'passive']

In [37]:
new_data.drop(diff_columns, axis=1, inplace=True)

In [38]:
new_data

,Наименование (device ID),Время отправки гео,bearing,Акселерометр,Гироскоп,Счетчик шагов,Скорость,"Точность, м",Долгота,Широта,Высота,Уровень заряда,timestamp_rownum,acc_x,acc_y,acc_z
0,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:35,274,"[9.241028, -1.1326599, -1.6559906]",[],21954,0.554510,29,37.613327,55.894135,179.2,60,1,9.241028,-1.132660,-1.655991
1,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:36,274,"[10.447708, -3.0320587, -3.1914673]",[],21954,0.554510,29,37.613327,55.894135,179.2,60,1,10.447708,-3.032059,-3.191467
2,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:36,274,"[7.4182434, -2.138214, -2.4093628]",[],21954,0.554510,29,37.613327,55.894135,179.2,60,2,7.418243,-2.138214,-2.409363
3,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:36,274,"[11.8810425, -1.5572205, -3.9480438]",[],21954,0.554510,29,37.613327,55.894135,179.2,60,3,11.881042,-1.557220,-3.948044
4,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:36,274,"[11.277695, -0.86450195, -2.7190094]",[],21957,0.554510,29,37.613327,55.894135,179.2,60,4,11.277695,-0.864502,-2.719009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98445,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:31,109,"[10.138062, -0.8421631, -1.3686829]",[],21952,0.549098,22,37.613377,55.894082,179.2,60,3,10.138062,-0.842163,-1.368683
98446,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:31,109,"[10.252975, -0.20370483, -1.2441864]",[],21952,0.549098,22,37.613377,55.894082,179.2,60,4,10.252975,-0.203705,-1.244186
98447,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:31,102,"[10.252975, -0.20370483, -1.2441864]",[],21952,0.401213,22,37.613391,55.894084,179.2,60,5,10.252975,-0.203705,-1.244186
98448,D4909E26-0469-4045-B426-211006C38915,2022-12-01 14:03:31,6,"[10.252975, -0.20370483, -1.2441864]",[],21952,0.777420,25,37.613397,55.894129,179.2,60,6,10.252975,-0.203705,-1.244186


In [39]:
new_data.columns

Index(['Наименование (device ID)', 'Время отправки гео', 'bearing',
       'Акселерометр', 'Гироскоп', 'Счетчик шагов', 'Скорость', 'Точность, м',
       'Долгота', 'Широта', 'Высота', 'Уровень заряда', 'timestamp_rownum',
       'acc_x', 'acc_y', 'acc_z'],
      dtype='object')

In [40]:
prev_data.columns

Index(['Наименование (device ID)', 'Время отправки гео', 'bearing',
       'Акселерометр', 'Гироскоп', 'Счетчик шагов', 'Скорость', 'Точность, м',
       'Долгота', 'Широта', 'Высота', 'Уровень заряда', 'timestamp_rownum',
       'acc_x', 'acc_y', 'acc_z'],
      dtype='object')

In [41]:
new_data.columns == prev_data.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [43]:
another_new_data = pd.read_parquet('../data/third_departure_parquet_preprocessed/raw/E9B9C64C-B66D-416D-BACD-3F0C9C2E1767/01.12.2022.parquet')

In [44]:
another_new_data

,userid,Наименование,Время создания записи,Время отправки гео,bearing,charge,isPermissionGranted,Акселерометр,Гироскоп,Счетчик шагов,...,passive,"Точность, м",Долгота,Широта,Высота,Уровень заряда,timestamp_rownum,acc_x,acc_y,acc_z
0,2897,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,2022-12-01 07:13:39.716,2022-12-01 07:13:18,300,False,True,"[7.673523, -4.7072906, 5.5149384]",[],141,...,True,15,37.599168,55.888997,188.70000,98,1,7.673523,-4.707291,5.514938
1,2897,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,2022-12-01 07:13:39.716,2022-12-01 07:13:18,300,False,True,"[7.600113, -3.8964539, 5.2467804]",[],141,...,True,15,37.599168,55.888997,188.70000,98,2,7.600113,-3.896454,5.246780
2,2897,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,2022-12-01 07:13:39.716,2022-12-01 07:13:18,300,False,True,"[8.066177, -4.0401154, 4.2922974]",[],141,...,True,15,37.599168,55.888997,188.70000,98,3,8.066177,-4.040115,4.292297
3,2897,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,2022-12-01 07:13:39.716,2022-12-01 07:13:19,300,False,True,"[8.666321, -3.826233, 3.4974213]",[],141,...,True,15,37.599168,55.888997,188.70000,98,1,8.666321,-3.826233,3.497421
4,2897,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,2022-12-01 07:13:39.716,2022-12-01 07:13:19,300,False,True,"[8.857864, -2.993042, 3.580429]",[],141,...,True,15,37.599168,55.888997,188.70000,98,2,8.857864,-2.993042,3.580429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154741,2897,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,2022-12-01 16:40:41.808,2022-12-01 16:40:39,40,False,True,"[-1.4116669, 1.8623962, 10.654495]",[],19122,...,True,14,37.599191,55.888982,190.29999,66,1,-1.411667,1.862396,10.654495
154742,2897,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,2022-12-01 16:40:41.808,2022-12-01 16:40:39,40,False,True,"[-1.4116669, 2.9733124, 8.585907]",[],19122,...,True,14,37.599191,55.888982,190.29999,66,2,-1.411667,2.973312,8.585907
154743,2897,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,2022-12-01 16:40:41.808,2022-12-01 16:40:39,40,False,True,"[-0.8753662, 2.7434692, 10.424652]",[],19122,...,True,14,37.599191,55.888982,190.29999,66,3,-0.875366,2.743469,10.424652
154744,2897,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,2022-12-01 16:40:41.808,2022-12-01 16:40:39,40,False,True,"[-0.18583679, 2.896698, 9.428665]",[],19122,...,True,14,37.599191,55.888982,190.29999,66,4,-0.185837,2.896698,9.428665


In [45]:
another_new_data.columns

Index(['userid', 'Наименование', 'Время создания записи', 'Время отправки гео',
       'bearing', 'charge', 'isPermissionGranted', 'Акселерометр', 'Гироскоп',
       'Счетчик шагов', 'Скорость', 'step_count', 'passive', 'Точность, м',
       'Долгота', 'Широта', 'Высота', 'Уровень заряда', 'timestamp_rownum',
       'acc_x', 'acc_y', 'acc_z'],
      dtype='object')

In [53]:
list_of_files_first = list()
for (dirpath, dirnames, filenames) in os.walk("../data/third_departure_parquet_preprocessed/raw/D4909E26-0469-4045-B426-211006C38915"):
    list_of_files_first += [os.path.join(dirpath, file) for file in filenames if ".parquet" in file]

In [54]:
list_of_files_first

['../data/third_departure_parquet_preprocessed/raw/D4909E26-0469-4045-B426-211006C38915\\01.12.2022.parquet',
 '../data/third_departure_parquet_preprocessed/raw/D4909E26-0469-4045-B426-211006C38915\\22.11.2022.parquet',
 '../data/third_departure_parquet_preprocessed/raw/D4909E26-0469-4045-B426-211006C38915\\23.11.2022.parquet',
 '../data/third_departure_parquet_preprocessed/raw/D4909E26-0469-4045-B426-211006C38915\\25.11.2022.parquet',
 '../data/third_departure_parquet_preprocessed/raw/D4909E26-0469-4045-B426-211006C38915\\28.11.2022.parquet',
 '../data/third_departure_parquet_preprocessed/raw/D4909E26-0469-4045-B426-211006C38915\\29.11.2022.parquet']

In [55]:
list_of_files_second = list()
for (dirpath, dirnames, filenames) in os.walk("../data/third_departure_parquet_preprocessed/raw/E9B9C64C-B66D-416D-BACD-3F0C9C2E1767"):
    list_of_files_second += [os.path.join(dirpath, file) for file in filenames if ".parquet" in file]

In [56]:
list_of_files_second

['../data/third_departure_parquet_preprocessed/raw/E9B9C64C-B66D-416D-BACD-3F0C9C2E1767\\01.12.2022.parquet',
 '../data/third_departure_parquet_preprocessed/raw/E9B9C64C-B66D-416D-BACD-3F0C9C2E1767\\22.11.2022.parquet',
 '../data/third_departure_parquet_preprocessed/raw/E9B9C64C-B66D-416D-BACD-3F0C9C2E1767\\23.11.2022.parquet',
 '../data/third_departure_parquet_preprocessed/raw/E9B9C64C-B66D-416D-BACD-3F0C9C2E1767\\25.11.2022.parquet',
 '../data/third_departure_parquet_preprocessed/raw/E9B9C64C-B66D-416D-BACD-3F0C9C2E1767\\29.11.2022.parquet']

In [52]:
def get_date_from_filename(filename):
    filename_list = filename.replace('.parquet', '').split("_")
    for each in filename_list:
        found_date = re.search("[0-9][0-9](.)[0-9][0-9](.)20[0-9][0-9]$", each)
        if found_date is not None:
            return found_date.group()

In [57]:
path_to_second_user = os.path.dirname(list_of_files_second[0])

In [58]:
path_to_second_user

'../data/third_departure_parquet_preprocessed/raw/E9B9C64C-B66D-416D-BACD-3F0C9C2E1767'

In [65]:
targets_df = []
for file_path in list_of_files_first:
    if os.path.join(path_to_second_user, (os.path.basename(file_path))) in list_of_files_second:
        second_df = pd.read_parquet(os.path.join(path_to_second_user, (os.path.basename(file_path))))
        initial_df = pd.read_parquet(file_path)
        second_df.rename(columns={'Наименование':'Наименование (device ID)'}, inplace=True)
        initial_df.rename(columns={'Наименование':'Наименование (device ID)'}, inplace=True)
        second_df.drop(diff_columns, axis=1, inplace=True)
        initial_df.drop(diff_columns, axis=1, inplace=True)
        date = initial_df["Время отправки гео"].dt.to_period("D").dt.to_timestamp()[0]
        second_name = second_df["Наименование (device ID)"][0]
        initial_name = initial_df["Наименование (device ID)"][0]
        tmp_df = pd.DataFrame([[date, second_name, initial_name]], columns=["date", "sec_name", "init_name"])
        targets_df.append(tmp_df)

In [67]:
targets_df = pd.concat(targets_df, ignore_index=True)

In [68]:
targets_df

,date,sec_name,init_name
0,2022-12-01,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,D4909E26-0469-4045-B426-211006C38915
1,2022-11-22,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,D4909E26-0469-4045-B426-211006C38915
2,2022-11-23,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,D4909E26-0469-4045-B426-211006C38915
3,2022-11-25,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,D4909E26-0469-4045-B426-211006C38915
4,2022-11-29,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,D4909E26-0469-4045-B426-211006C38915


In [71]:
def count_speed(row):
    x_diff_1 = row["x_metres_x_left"] - row["x_metres_x_right"]
    y_diff_1 = row["y_metres_x_left"] - row["y_metres_x_right"]
    x_diff_2 = row["x_metres_y_left"] - row["x_metres_y_right"]
    y_diff_2 = row["y_metres_y_left"] - row["y_metres_y_right"]
    time_diff = (row["Время отправки гео_left"] - row["Время отправки гео_right"]).total_seconds()

    distance_1 = np.sqrt(x_diff_1 ** 2 + y_diff_1 ** 2)
    distance_2 = np.sqrt(x_diff_2 ** 2 + y_diff_2 ** 2)
    row["speed_left"] = distance_1 / time_diff
    row["speed_right"] = distance_2 / time_diff
    return row


def count_correlation_features(merged_df: gpd.GeoDataFrame,
                               data_df: pd.DataFrame):
    accelerometer_axis = ["x", "y", "z"]
    for axis in accelerometer_axis:
        data_df[f"accelerometer_{axis}_corr"] = abs(merged_df[[f"acc_{axis}_x", f"acc_{axis}_y"]].corr().values[1, 0])
        data_df[f"accelerometer_{axis}_diff_corr"] = abs(merged_df[[f"acc_{axis}_x", f"acc_{axis}_y"]].diff(1).corr().values[1, 0])

    metres_columns_correlation = ["x", "y"]
    for meter_column in metres_columns_correlation:
        data_df[f"{meter_column}_metres_corr"] = abs(merged_df[[f"{meter_column}_metres_x", f"{meter_column}_metres_y"]].corr().values[1, 0])
        data_df[f"{meter_column}_diff_metres_corr"] = abs(merged_df[[f"{meter_column}_metres_x", f"{meter_column}_metres_y"]].diff(1).corr().values[1, 0])

    data_df["height_corr"] = abs(merged_df[["Высота_x", "Высота_y"]].corr().values[1, 0])
    data_df["bearing_corr"] = abs(merged_df[["bearing_x", "bearing_y"]].corr().values[1, 0])
    data_df["speed_corr"] = abs(merged_df[["Скорость_x", "Скорость_y"]].corr().values[1, 0])

    data_df["height_diff_corr"] = abs(merged_df[["Высота_x", "Высота_y"]].diff(1).corr().values[1, 0])
    data_df["bearing_diff_corr"] = abs(merged_df[["bearing_x", "bearing_y"]].diff(1).corr().values[1, 0])
    data_df["speed_diff_corr"] = abs(merged_df[["Скорость_x", "Скорость_y"]].diff(1).corr().values[1, 0])

    merged_itself = merged_df.join(merged_df.shift(), lsuffix="_left", rsuffix="_right")
    merged_itself.reset_index(drop=True, inplace=True)
    merged_itself = merged_itself.loc[1:].apply(lambda x: count_speed(x), axis=1)

    data_df["speed_corr"] = abs(merged_itself[["speed_left", "speed_right"]].corr().values[1, 0])
    data_df["speed_diff_corr"] = abs(merged_itself[["speed_left", "speed_right"]].diff(1).corr().values[1, 0])

    data_df["speed_mean_abs_diff"] = abs(merged_itself["speed_left"] - merged_itself["speed_right"]).mean()
    data_df["speed_mean_diff"] = (merged_itself["speed_left"] - merged_itself["speed_right"]).mean()
    data_df["speed_std_diff"] = (merged_itself["speed_left"] - merged_itself["speed_right"]).std()
    return data_df


def get_features_from_dataframes(gdf1: gpd.GeoDataFrame,
                                 gdf2: gpd.GeoDataFrame):
    data = pd.DataFrame()
    merged_df = gdf1.merge(gdf2, on=["Время отправки гео"], how="inner")
    data["date"] = [gdf1["Время отправки гео"].dt.to_period('D').values[0].to_timestamp()]
    data["datetime"] = gdf1.reset_index()["time_category"][0].left
    data["dev_id_1"] = gdf1["Наименование (device ID)"].values[0]
    data["id_1_last_geometry"] = gdf1.tail(1).reset_index()["geometry"][0]
    data["id_1_last_lon"] = gdf1.tail(1).reset_index()["Долгота"][0]
    data["id_1_last_lat"] = gdf1.tail(1).reset_index()["Широта"][0]
    data["dev_id_2"] = gdf2["Наименование (device ID)"].values[0]
    data["id_2_last_geometry"] = gdf2.tail(1).reset_index()["geometry"][0]
    data["id_2_last_lon"] = gdf2.tail(1).reset_index()["Долгота"][0]
    data["id_2_last_lat"] = gdf2.tail(1).reset_index()["Широта"][0]
    if merged_df.shape[0] < 5:
        return data

    merged_df['distance'] = merged_df.apply(lambda x: x["geometry_x"].distance(x["geometry_y"]), axis=1)

    data = count_correlation_features(merged_df, data)
    data["distance_mean"] = merged_df['distance'].mean()
    data["distance_std"] = merged_df['distance'].std()
    return data

In [106]:
for file in list_of_files_first+list_of_files_second:
    temp_data = pd.read_parquet(file)
    temp_data.rename(columns={'Наименование':'Наименование (device ID)'}, inplace=True)
    temp_data.drop(diff_columns, axis=1, inplace=True)
    os.makedirs(f'../data/third_departure_parquet_preprocessed/prep/{os.path.basename(os.path.dirname(file))}',exist_ok=True)
    temp_data.to_parquet(f'../data/third_departure_parquet_preprocessed/prep/{os.path.basename(os.path.dirname(file))}/{os.path.basename(file)}')

In [107]:
new_list_of_files_first = list()
for (dirpath, dirnames, filenames) in os.walk("../data/third_departure_parquet_preprocessed/prep/D4909E26-0469-4045-B426-211006C38915"):
    new_list_of_files_first += [os.path.join(dirpath, file) for file in filenames if ".parquet" in file]

In [113]:
new_list_of_files_second = list()
for (dirpath, dirnames, filenames) in os.walk("../data/third_departure_parquet_preprocessed/prep/E9B9C64C-B66D-416D-BACD-3F0C9C2E1767"):
    new_list_of_files_second += [os.path.join(dirpath, file) for file in filenames if ".parquet" in file]

In [114]:
dict_dates = dict()
for filename in tqdm(new_list_of_files_first+new_list_of_files_second):
    date = get_date_from_filename(filename)
    if date not in dict_dates:
        dict_dates[date] = []
    df = pd.read_parquet(filename)
    dict_dates[date].append(df)

100%|██████████| 11/11 [00:00<00:00, 14.71it/s]


In [115]:
dict_dates

{'01.12.2022': [                   Наименование (device ID)  Время отправки гео  bearing  \
  0      D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:35      274   
  1      D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:36      274   
  2      D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:36      274   
  3      D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:36      274   
  4      D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:36      274   
  ...                                     ...                 ...      ...   
  98445  D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:31      109   
  98446  D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:31      109   
  98447  D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:31      102   
  98448  D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:31        6   
  98449  D4909E26-0469-4045-B426-211006C38915 2022-12-01 14:03:31      274   
  
                                 Акселерометр Г

In [79]:
def make_subsample_for_df(df):
    subsample = df.sample(n=50)
    dev_id = subsample["Наименование (device ID)"].values[0]
    gdf_subsample = gpd.GeoDataFrame(subsample,
                                     geometry=gpd.points_from_xy(subsample['Долгота'], subsample['Широта']),
                                     crs=4326).to_crs(3857)
    data_for_gdf = d = {"dev_id": [dev_id], "geometry":[gdf_subsample.buffer(1).unary_union]}
    output_gdf = gpd.GeoDataFrame(data_for_gdf, crs=3857)
    return output_gdf

In [80]:
def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    dt_column = "Время отправки гео"
    device_id_column = "Наименование (device ID)"
    if not df[dt_column].is_monotonic_increasing:
        df = df.sort_values(by=[dt_column], ignore_index=True)
    df = df.groupby([device_id_column, dt_column]).head(1)
    df = df.sort_values(by=[dt_column], ignore_index=True)
    return df

In [81]:
def convert_dataframe_to_geodataframe(df: pd.DataFrame) -> gpd.GeoDataFrame:
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Широта'], df['Долгота']),
                                     crs=4326).to_crs(32633)
    gdf["x_metres"] = gdf["geometry"].x
    gdf["y_metres"] = gdf["geometry"].y
    return gdf

In [82]:
def make_subsample_for_df(df):
    subsample = df.sample(n=50)
    dev_id = subsample["Наименование (device ID)"].values[0]
    gdf_subsample = gpd.GeoDataFrame(subsample,
                                     geometry=gpd.points_from_xy(subsample['Долгота'], subsample['Широта']),
                                     crs=4326).to_crs(3857)
    data_for_gdf = d = {"dev_id": [dev_id], "geometry":[gdf_subsample.buffer(1).unary_union]}
    output_gdf = gpd.GeoDataFrame(data_for_gdf, crs=32633)
    return output_gdf

In [83]:
def find_possible_intersections(df: gpd.GeoDataFrame):
    df_copy = df.copy(deep=True)
    df_copy["geometry"] = df_copy.buffer(50)
    data_for_gdf = {"dev_id": [],
                    "geometry": []}
    for unique_id in df_copy["Наименование (device ID)"].unique():
        data_for_gdf["dev_id"].append(unique_id)
        data_for_gdf["geometry"].append(df_copy[df_copy["Наименование (device ID)"] == unique_id].unary_union)
    batch_polygons = gpd.GeoDataFrame(data_for_gdf, crs=32633)
    possible_intersects = batch_polygons.sjoin(batch_polygons, how='inner', predicate="intersects")
    possible_intersects = possible_intersects[possible_intersects["dev_id_left"] !=
                                              possible_intersects["dev_id_right"]]
    return possible_intersects

In [84]:
def drop_permutations_copys(df: pd.DataFrame) -> pd.DataFrame:
    cols = df.columns.values
    df.loc[:, cols] = np.sort(df.loc[:, cols].values, axis=1)
    df = df.drop_duplicates()
    return df

In [85]:
def count_speed(row):
    x_diff_1 = row["x_metres_x_left"] - row["x_metres_x_right"]
    y_diff_1 = row["y_metres_x_left"] - row["y_metres_x_right"]
    x_diff_2 = row["x_metres_y_left"] - row["x_metres_y_right"]
    y_diff_2 = row["y_metres_y_left"] - row["y_metres_y_right"]
    time_diff = (row["Время отправки гео_left"] - row["Время отправки гео_right"]).total_seconds()

    distance_1 = np.sqrt(x_diff_1 ** 2 + y_diff_1 ** 2)
    distance_2 = np.sqrt(x_diff_2 ** 2 + y_diff_2 ** 2)
    row["speed_left"] = distance_1 / time_diff
    row["speed_right"] = distance_2 / time_diff
    return row


def count_correlation_features(merged_df: gpd.GeoDataFrame,
                               data_df: pd.DataFrame):
    accelerometer_axis = ["x", "y", "z"]
    for axis in accelerometer_axis:
        data_df[f"accelerometer_{axis}_corr"] = abs(merged_df[[f"acc_{axis}_x", f"acc_{axis}_y"]].corr().values[1, 0])
        data_df[f"accelerometer_{axis}_diff_corr"] = abs(merged_df[[f"acc_{axis}_x", f"acc_{axis}_y"]].diff(1).corr().values[1, 0])

    metres_columns_correlation = ["x", "y"]
    for meter_column in metres_columns_correlation:
        data_df[f"{meter_column}_metres_corr"] = abs(merged_df[[f"{meter_column}_metres_x", f"{meter_column}_metres_y"]].corr().values[1, 0])
        data_df[f"{meter_column}_diff_metres_corr"] = abs(merged_df[[f"{meter_column}_metres_x", f"{meter_column}_metres_y"]].diff(1).corr().values[1, 0])

    data_df["height_corr"] = abs(merged_df[["Высота_x", "Высота_y"]].corr().values[1, 0])
    data_df["bearing_corr"] = abs(merged_df[["bearing_x", "bearing_y"]].corr().values[1, 0])
    data_df["speed_corr"] = abs(merged_df[["Скорость_x", "Скорость_y"]].corr().values[1, 0])

    data_df["height_diff_corr"] = abs(merged_df[["Высота_x", "Высота_y"]].diff(1).corr().values[1, 0])
    data_df["bearing_diff_corr"] = abs(merged_df[["bearing_x", "bearing_y"]].diff(1).corr().values[1, 0])
    data_df["speed_diff_corr"] = abs(merged_df[["Скорость_x", "Скорость_y"]].diff(1).corr().values[1, 0])

    merged_itself = merged_df.join(merged_df.shift(), lsuffix="_left", rsuffix="_right")
    merged_itself.reset_index(drop=True, inplace=True)
    merged_itself = merged_itself.loc[1:].apply(lambda x: count_speed(x), axis=1)

    data_df["speed_corr"] = abs(merged_itself[["speed_left", "speed_right"]].corr().values[1, 0])
    data_df["speed_diff_corr"] = abs(merged_itself[["speed_left", "speed_right"]].diff(1).corr().values[1, 0])

    data_df["speed_mean_abs_diff"] = abs(merged_itself["speed_left"] - merged_itself["speed_right"]).mean()
    data_df["speed_mean_diff"] = (merged_itself["speed_left"] - merged_itself["speed_right"]).mean()
    data_df["speed_std_diff"] = (merged_itself["speed_left"] - merged_itself["speed_right"]).std()
    return data_df


def get_features_from_dataframes(gdf1: gpd.GeoDataFrame,
                                 gdf2: gpd.GeoDataFrame):
    data = pd.DataFrame()
    merged_df = gdf1.merge(gdf2, on=["Время отправки гео"], how="inner")
    data["date"] = [gdf1["Время отправки гео"].dt.to_period('D').values[0].to_timestamp()]
    data["datetime"] = gdf1.reset_index()["time_category"][0].left
    data["dev_id_1"] = gdf1["Наименование (device ID)"].values[0]
    data["id_1_last_geometry"] = gdf1.tail(1).reset_index()["geometry"][0]
    data["id_1_last_lon"] = gdf1.tail(1).reset_index()["Долгота"][0]
    data["id_1_last_lat"] = gdf1.tail(1).reset_index()["Широта"][0]
    data["dev_id_2"] = gdf2["Наименование (device ID)"].values[0]
    data["id_2_last_geometry"] = gdf2.tail(1).reset_index()["geometry"][0]
    data["id_2_last_lon"] = gdf2.tail(1).reset_index()["Долгота"][0]
    data["id_2_last_lat"] = gdf2.tail(1).reset_index()["Широта"][0]
    if merged_df.shape[0] < 5:
        return data

    merged_df['distance'] = merged_df.apply(lambda x: x["geometry_x"].distance(x["geometry_y"]), axis=1)

    data = count_correlation_features(merged_df, data)
    data["distance_mean"] = merged_df['distance'].mean()
    data["distance_std"] = merged_df['distance'].std()
    return data

In [147]:
dates = dict_dates.keys()
feats = []
day_feats = []
for date in tqdm(dates):
    feats_per_day = []
    data_list = dict_dates[date]
    all_data = pd.concat(data_list, ignore_index=True)
    all_data_preprocessd = preprocess_dataframe(all_data)
    all_data_gdf = convert_dataframe_to_geodataframe(all_data_preprocessd)
    cuts = pd.date_range(all_data_gdf["Время отправки гео"].min() - pd.Timedelta("5min"),
                     all_data_gdf["Время отправки гео"].max() + pd.Timedelta("5min"),
                     freq="5min")
    all_data_gdf["time_category"] = pd.cut(all_data_gdf["Время отправки гео"], cuts)
    useful_samples = []
    for each in all_data_gdf["time_category"].unique():
        tmp_df = all_data_gdf[all_data_gdf["time_category"] == each]
        if tmp_df["Наименование (device ID)"].nunique() > 2:
            useful_samples.append(tmp_df)
    for sample in useful_samples:
        possible_intersecions = find_possible_intersections(sample)
        relevant_pairs = possible_intersecions[["dev_id_left", "dev_id_right"]]
        relevant_pairs = drop_permutations_copys(relevant_pairs)
        for i, (first_id, second_id) in relevant_pairs.iterrows():
            df1 = sample[sample["Наименование (device ID)"] == first_id]
            df2 = sample[sample["Наименование (device ID)"] == second_id]
            feats.append(get_features_from_dataframes(df1, df2))
            feats_per_day.append(get_features_from_dataframes(df1, df2))

    day_feats.append(feats_per_day)
        # break
    # break
    # print("test")


100%|██████████| 6/6 [00:06<00:00,  1.16s/it]


In [148]:
day_feats

[[], [], [], [], [], []]